<a href="https://colab.research.google.com/github/ElleuchMed2022/ML-programming-/blob/main/Text_Classification_using_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string



In [2]:
import numpy as np 
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [3]:
import gensim.downloader as api 
glove_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
import pandas as pd 

In [5]:
from google.colab import files
uploaded = files.upload()

Saving data_5_column.xlsx to data_5_column.xlsx


In [6]:
import io 
df = pd.read_excel(io.BytesIO(uploaded["data_5_column.xlsx"]))
df.head(5)

,Unnamed: 0,Poste,Entreprise,Contrat,description,Localisation,salaire €
0,0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000
1,1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000
2,2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000
3,3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000
4,4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000


In [7]:
del df["Unnamed: 0"]

In [8]:
df.head(5)


,Poste,Entreprise,Contrat,description,Localisation,salaire €
0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000
1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000
2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000
3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000
4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000


In [9]:
# Creates list of texts and labels

text = df['description'].values.tolist()

labels = df['salaire €'].values.tolist()

In [10]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
labels = LE.fit_transform(labels)

In [11]:
data = df

In [12]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['clean_description'] = data['description'].apply(lambda x: remove_punct(x))

In [13]:
data

,Poste,Entreprise,Contrat,description,Localisation,salaire €,clean_description
0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000,Notre agence ARTUS INTERIM de Vannes recherche...
1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000,En tant que Mécanicien PL et rattaché au Chef ...
2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000,Manpower SABLE SUR SARTHE recherche pour son c...
3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000,Dans le cadre du développement constant de son...
4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000,MENCO ST NAZAIRE recrute des Électriciens conf...
...,...,...,...,...,...,...,...
4458,MEDECIN SPECIALISE EN PEDOPSYCHIATRIE H/F,UGECAM OCCITANIE,CDI,Contexte\nLe Centre Médico Educatif de l’Enfan...,Montpellier (34),43000-90000,Contexte\nLe Centre Médico Educatif de l’Enfan...
4459,Médecin DIM (H/F),UGECAM AQUITAINE,CDI,"Contexte\nL’UGECAM Aquitaine, organisme à but ...",Bruges (33),30000-43000,Contexte\nL’UGECAM Aquitaine organisme à but n...
4460,Médecin ophtalmologue médical /chirurgical H/F,HAYS,"Temps plein, CDI, CDD",Mettez votre savoir faire au profit d'un établ...,Lyon (69),90000-350000,Mettez votre savoir faire au profit dun établi...
4461,Médecin Généraliste H/F - Paris 11 Richard Lenoir,Expert et Santé,CDI,CD - IDF\nNous recherchons pour nos partenaire...,Paris (75),90000-350000,CD IDF\nNous recherchons pour nos partenaires...


In [14]:
!pip install nltk

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.clean_description] 



In [17]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('french')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [20]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [21]:
result = [' '.join(sen) for sen in filtered_words] 

In [22]:


data['description_Final'] = result


In [23]:
data

,Poste,Entreprise,Contrat,description,Localisation,salaire €,clean_description,description_Final
0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000,Notre agence ARTUS INTERIM de Vannes recherche...,agence artus interim vannes recherche lun clie...
1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000,En tant que Mécanicien PL et rattaché au Chef ...,tant mécanicien pl rattaché chef datelier diff...
2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000,Manpower SABLE SUR SARTHE recherche pour son c...,manpower sable sarthe recherche client acteur ...
3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000,Dans le cadre du développement constant de son...,cadre développement constant activité garantir...
4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000,MENCO ST NAZAIRE recrute des Électriciens conf...,menco st nazaire recrute électriciens confirmé...
...,...,...,...,...,...,...,...,...
4458,MEDECIN SPECIALISE EN PEDOPSYCHIATRIE H/F,UGECAM OCCITANIE,CDI,Contexte\nLe Centre Médico Educatif de l’Enfan...,Montpellier (34),43000-90000,Contexte\nLe Centre Médico Educatif de l’Enfan...,contexte centre médico educatif ’ enfance font...
4459,Médecin DIM (H/F),UGECAM AQUITAINE,CDI,"Contexte\nL’UGECAM Aquitaine, organisme à but ...",Bruges (33),30000-43000,Contexte\nL’UGECAM Aquitaine organisme à but n...,contexte ’ ugecam aquitaine organisme but non ...
4460,Médecin ophtalmologue médical /chirurgical H/F,HAYS,"Temps plein, CDI, CDD",Mettez votre savoir faire au profit d'un établ...,Lyon (69),90000-350000,Mettez votre savoir faire au profit dun établi...,mettez savoir faire profit dun établissement d...
4461,Médecin Généraliste H/F - Paris 11 Richard Lenoir,Expert et Santé,CDI,CD - IDF\nNous recherchons pour nos partenaire...,Paris (75),90000-350000,CD IDF\nNous recherchons pour nos partenaires...,cd idf recherchons partenaires médecin général...


In [24]:
data['tokens'] = filtered_words

In [25]:
data.head(5)

,Poste,Entreprise,Contrat,description,Localisation,salaire €,clean_description,description_Final,tokens
0,Tuyauteur (H/F),ARTUS INTERIM VANNES,Intérim,Notre agence ARTUS INTERIM de Vannes recherche...,\n ARTUS INTERI...,23000-30000,Notre agence ARTUS INTERIM de Vannes recherche...,agence artus interim vannes recherche lun clie...,"[agence, artus, interim, vannes, recherche, lu..."
1,MECANICIEN PL (H/F),SOC GARAGE VEHICULES INDUSTRIELS,CDI,En tant que Mécanicien PL et rattaché au Chef ...,Évreux,23000-30000,En tant que Mécanicien PL et rattaché au Chef ...,tant mécanicien pl rattaché chef datelier diff...,"[tant, mécanicien, pl, rattaché, chef, datelie..."
2,Opérateur de production (H/F),Manpower SABLE SUR SARTHE,Intérim 3 mois,Manpower SABLE SUR SARTHE recherche pour son c...,NaN,23000-30000,Manpower SABLE SUR SARTHE recherche pour son c...,manpower sable sarthe recherche client acteur ...,"[manpower, sable, sarthe, recherche, client, a..."
3,TECHNICIEN ELECTROMECA SAV H/F,DE GRAËT CONSULTING|DE GRAET CONSULTING,CDI,Dans le cadre du développement constant de son...,NaN,23000-30000,Dans le cadre du développement constant de son...,cadre développement constant activité garantir...,"[cadre, développement, constant, activité, gar..."
4,ELECTRICIEN H/F,MENCO SAINT NAZAIRE,Intérim,"""MENCO ST NAZAIRE recrute des Électriciens con...",\n FINANCIERE M...,5000-23000,MENCO ST NAZAIRE recrute des Électriciens conf...,menco st nazaire recrute électriciens confirmé...,"[menco, st, nazaire, recrute, électriciens, co..."


In [26]:


data = data[['description_Final', 'tokens', 'salaire €']]



In [27]:
data

,description_Final,tokens,salaire €
0,agence artus interim vannes recherche lun clie...,"[agence, artus, interim, vannes, recherche, lu...",23000-30000
1,tant mécanicien pl rattaché chef datelier diff...,"[tant, mécanicien, pl, rattaché, chef, datelie...",23000-30000
2,manpower sable sarthe recherche client acteur ...,"[manpower, sable, sarthe, recherche, client, a...",23000-30000
3,cadre développement constant activité garantir...,"[cadre, développement, constant, activité, gar...",23000-30000
4,menco st nazaire recrute électriciens confirmé...,"[menco, st, nazaire, recrute, électriciens, co...",5000-23000
...,...,...,...
4458,contexte centre médico educatif ’ enfance font...,"[contexte, centre, médico, educatif, ’, enfanc...",43000-90000
4459,contexte ’ ugecam aquitaine organisme but non ...,"[contexte, ’, ugecam, aquitaine, organisme, bu...",30000-43000
4460,mettez savoir faire profit dun établissement d...,"[mettez, savoir, faire, profit, dun, établisse...",90000-350000
4461,cd idf recherchons partenaires médecin général...,"[cd, idf, recherchons, partenaires, médecin, g...",90000-350000


In [28]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
labels= data["salaire €"]
labels = LE.fit_transform(labels)

In [29]:
data["salaire €"]= labels

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [177]:


all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))



533476 words total, with a vocabulary size of 36274
Max sentence length is 1424


In [178]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

57840 words total, with a vocabulary size of 11232
Max sentence length is 617


In [179]:
word2vec = glove_model

In [180]:


def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

In [181]:
def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [182]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [189]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300


In [190]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["description_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["description_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 36246 unique tokens.


In [191]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [192]:


train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)



(36247, 300)


In [193]:
test_sequences = tokenizer.texts_to_sequences(data_test["description_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [194]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)
    
    x = Dropout(0.3)(l_merge)  
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
   
    
    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model


In [195]:
label_names = ['salaire €']

In [196]:
y_train = data_train[label_names].values

In [197]:
y_train

array([[2],
       [3],
       [2],
       ...,
       [4],
       [2],
       [2]])

In [198]:
x_train = train_cnn_data
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_tr = y_train

In [199]:
x_train

array([[ 7316, 20364,   739, ...,   304,   335,  4119],
       [ 2896,   296,  9276, ...,    32,    75,    87],
       [   25,     1,     9, ...,   940,     1,  1570],
       ...,
       [  901,   306,     8, ...,   143,   411,  1364],
       [  538,  7613,   926, ...,   758,  2407,  1179],
       [    1,   115,   528, ...,    69,   808,   678]], dtype=int32)

In [200]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 5 )


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 embedding_19 (Embedding)       (None, 50, 300)      10874100    ['input_20[0][0]']               
                                                                                                  
 conv1d_95 (Conv1D)             (None, 49, 200)      120200      ['embedding_19[0][0]']           
                                                                                                  
 conv1d_96 (Conv1D)             (None, 48, 200)      180200      ['embedding_19[0][0]']           
                                                                                            

In [167]:


num_epochs = 5
batch_size = 16



In [168]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/5
226/226 [==============================] - 24s 103ms/step - loss: 1.3544 - acc: 0.4402 - val_loss: 1.0076 - val_acc: 0.5274
Epoch 2/5
226/226 [==============================] - 24s 104ms/step - loss: 0.9144 - acc: 0.6079 - val_loss: 0.9035 - val_acc: 0.6119
Epoch 3/5
226/226 [==============================] - 22s 99ms/step - loss: 0.6756 - acc: 0.7133 - val_loss: 0.8005 - val_acc: 0.6667
Epoch 4/5
226/226 [==============================] - 22s 99ms/step - loss: 0.4960 - acc: 0.8041 - val_loss: 0.8013 - val_acc: 0.6766
Epoch 5/5
226/226 [==============================] - 22s 98ms/step - loss: 0.3626 - acc: 0.8683 - val_loss: 0.7177 - val_acc: 0.7388


In [171]:
output_test = model.predict(test_cnn_data)

array([[9.16352093e-01, 7.39875855e-03, 2.99230072e-04, 7.52455592e-02,
        7.04414677e-04],
       [5.32713693e-06, 1.29149295e-04, 9.99591649e-01, 3.14888234e-06,
        2.70683144e-04],
       [9.72761631e-01, 2.30123452e-03, 9.66136286e-05, 2.47155055e-02,
        1.25018385e-04],
       ...,
       [5.68839252e-01, 2.62827929e-02, 5.00490237e-03, 3.92013311e-01,
        7.85976090e-03],
       [9.46563244e-01, 3.36266570e-02, 1.10719493e-03, 9.70217492e-03,
        9.00086854e-03],
       [8.56722534e-01, 2.21833345e-02, 5.60727378e-04, 1.18848756e-01,
        1.68474379e-03]], dtype=float32)

In [174]:
y_test = data_test["salaire €"]

In [175]:
m= []
for i in range( len(output_test)): 
  m.append(output_test[i].argmax())

In [176]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,m))

0.7203579418344519
